In [1]:
import torch
import torch.onnx
!pip install onnx
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.4 MB/s eta 0:00:00


In [2]:
# Создание модели RNN
model = torch.nn.RNN(10, 20, 2)
dummy_input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
output, hn = model(dummy_input, h0)
print(output)

Результат запуска модели RNN в PyTorch: tensor([[[-4.7851e-02, -6.5674e-01,  9.6824e-01, -2.6246e-01, -8.3156e-01,
          -1.0074e-01, -1.9447e-01,  5.7331e-01,  3.2693e-01, -5.7320e-02,
          -2.6907e-01,  5.1821e-01,  1.6294e-01, -6.2389e-01, -2.3291e-01,
          -7.2474e-01,  5.3956e-01,  4.2081e-01,  4.5860e-01,  8.8232e-01],
         [-1.6344e-01, -7.3212e-01,  8.8238e-01,  3.4850e-01,  1.5741e-01,
          -3.5036e-01, -3.9629e-01,  3.3504e-01,  7.0943e-01, -2.3102e-01,
          -4.3995e-01, -7.1831e-01,  8.4732e-01,  1.1262e-01, -1.5322e-01,
           5.4671e-01,  5.6726e-01,  8.3800e-01,  6.0213e-01,  3.7237e-01],
         [ 6.6188e-01, -5.3462e-01,  4.2542e-02,  2.1600e-01,  7.4994e-01,
          -5.5081e-01,  7.0632e-01, -3.5615e-01,  5.9041e-01, -2.8182e-01,
          -3.0207e-01,  3.5385e-01, -2.8103e-01, -7.6550e-01,  7.0553e-02,
          -4.0636e-01, -4.8250e-01, -4.2536e-01, -1.1637e-01,  6.9370e-01]],

        [[ 7.7240e-02,  1.3621e-01,  6.1979e-01,  3.807

In [3]:
# Конвертация в ONNX
torch.onnx.export(model, dummy_input, "rnn.onnx")

/usr/local/lib/python3.10/dist-packages/torch/onnx/symbolic_opset9.py:4661: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with RNN_TANH can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


In [4]:
# Загрузка и запуск модели в ONNX
import onnx
import onnxruntime
onnx_model = onnx.load("rnn.onnx")
ort_session = onnxruntime.InferenceSession("rnn.onnx")
ort_inputs = {ort_session.get_inputs()[0].name: dummy_input.numpy()}
ort_outs = ort_session.run(None, ort_inputs)
print(ort_outs)

Результат запуска модели RNN в ONNX: [array([[[ 2.17823267e-01, -2.03199863e-01,  3.38312387e-01,
         -6.11491799e-02, -2.14505434e-01,  3.85571718e-01,
          1.07080340e-01,  6.60947561e-02,  1.49716139e-01,
          8.08495283e-02, -2.61459947e-02,  2.81213522e-02,
         -2.34387517e-02, -2.29871631e-01,  1.42533064e-01,
          1.30758524e-01,  3.51597309e-01, -1.71777666e-01,
         -9.35329795e-02,  3.81598115e-01],
        [ 3.02509904e-01, -2.18734741e-01,  4.10835743e-01,
          3.11330557e-02, -5.22601485e-01,  4.85967278e-01,
         -2.24854529e-01,  8.16391706e-02,  3.42656612e-01,
         -2.60089695e-01,  2.93462515e-01, -1.44952595e-01,
          4.80427742e-02,  3.35390568e-02, -5.27411163e-01,
         -7.60433674e-02,  1.40766263e-01, -2.97369778e-01,
          9.51350927e-02,  3.74128938e-01],
        [ 5.04579425e-01, -3.71215999e-01,  6.50324106e-01,
         -1.42696083e-01,  3.63821983e-02,  2.72772193e-01,
          4.37738538e-01,  2.29078

In [5]:
# Обратная конвертация и запуск модели
!pip install onnx2torch
from onnx2torch.converter import convert
torch_model = convert("rnn.onnx")
output = torch_model(dummy_input)
print(output)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.1/80.1 kB 2.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

NotImplementedError: Converter is not implemented (OperationDescription(domain='', operation_type='RNN', version=14))